In [1]:
import sys
sys.path.insert(0, '..')

import numpy as np
from activation_func.relu import Activation_ReLU
from layers.layer_dense import Layer_Dense

In [2]:
#random thinking about matrix multiplications lol 
x=np.random.randn(2,784)
layer1=Layer_Dense(784,200)
relu=Activation_ReLU()
layer1.forward(x,training=True)
relu.forward(layer1.output,training=True)

#padding is adding zeroes to the image matrix white it is being processed
So output matrix will have more knowledge about the middle part of the image than edges and we might lose some important features from edges.
so two downsides are shrinking and edges are not used a lot.  

In [3]:
#zero padding for height and width dimensions     
#shape out == [n+2*pad] for given dims which are padded
def zero_padding(inputs,padding):
    out=np.pad(inputs,((0,0),(padding,padding),(padding,padding),(0,0)),mode='constant',constant_values=(0,0))
    return out

In [4]:
#out shape should be  
images=np.random.randn(4,6,6,3)
padding=0
out=zero_padding(images,padding)
#compare the shapes [n+2p]
print('shape after padding',out.shape)
print('shape match ',out.shape[1]==images.shape[1]+(2*padding))

shape after padding (4, 6, 6, 3)
shape match  True


In [5]:
#3D convolution example with one filter and relu activation function
image=np.random.rand(6,6,3)
W1=np.random.rand(3,3,3)
relu =Activation_ReLU()

(n_h_prev,n_w_prev,n_c_prev)=image.shape
(f,f,n_c_prev)=W1.shape

#output shape [N-F+1]
n_h=int(n_h_prev-f+1)
n_w=int(n_w_prev-f+1)

#init output with zeroes  
output=np.zeros((n_h,n_w))

for h in range(n_h):
    for w in range(n_w):
        #define vert-horiz start end for convolution
        vert_start=h
        vert_end=vert_start+f
        horiz_start=w
        horiz_end=w+f
        #slice of the image to perform convolution 
        a_prev=image[vert_start:vert_end,horiz_start:horiz_end,:]
        #convolution and store value 
        output[h,w]=np.sum(np.multiply(W1,a_prev))

relu.forward(output,training=True)
print(output)
print('relu output shape',relu.output.shape)

[[4.91085472 5.02681643 6.136079   5.99971182]
 [6.148262   6.46462906 7.33183787 6.53712559]
 [5.9483094  7.94525324 7.37590482 6.79068827]
 [7.12852876 6.57504757 6.82846091 7.36506999]]
relu output shape (4, 4)


In [6]:
#2D convolution example with one filter 
image=np.random.rand(4,4)
filter=np.random.rand(3,3)

#shapes
(n_h_prev,n_w_prev)=image.shape
(f,f)=filter.shape

#output shape
n_h=int(n_h_prev-f+1)
n_w=int(n_w_prev-f+1)

#output
output=np.zeros((n_h,n_w))

for h in range(n_h):
    for w in range(n_w):
        #vertical/horizontal start and end
        vert_start=h
        vert_end=vert_start+f
        horiz_start=w
        horiz_end=horiz_start+f

        image_slice=image[vert_start:vert_end,horiz_start:horiz_end]
        output[h,w]=np.sum(np.multiply(image_slice,filter))

print('output shape',output.shape)
print('output values',output)

output shape (2, 2)
output values [[1.59919375 1.62622864]
 [1.17751356 2.18563288]]


In [13]:
#one layer of convolution
#3d convolution with batch of images and multiple filters/bias  and relu act/func

def conv_forward(inputs,w1,b1,padding,stride):
        
    #get shapes
    (m,n_h_prev,n_w_prev,n_c_prev)=inputs.shape
    (f,f,n_c_f_prev,n_c)=w1.shape

    #define output shape
    n_h=int((n_h_prev-f+2*padding)/stride)+1
    n_w=int((n_w_prev-f+2*padding)/stride)+1

    #init output tensor 
    output=np.zeros((m,n_h,n_w,n_c)) 

    #padding 
    inputs_pad=zero_padding(inputs,padding)

    for i in range(m):
        inputi=inputs_pad[i]
        for h in range(n_h):
            for w in range(n_w):
                for c in range(n_c):
                    #define vertical/horizontal start-end 
                    vert_start=h*stride
                    vert_end=vert_start+f
                    horiz_start=w*stride
                    horiz_end=horiz_start+f
                    
                    #get image slice
                    input_slice=inputi[vert_start:vert_end,horiz_start:horiz_end,:]
                    #convolution and store to a new tensor values
                    output[i,h,w,c]=np.sum(np.multiply(input_slice,w1[...,c]))+b1[...,c]

    return output 

In [29]:
from activation_func.softmax import Activation_Softmax

In [32]:
#simple convolution network example:conv1,conv2,conv3,softmax
inputs=np.random.rand(3,39,39,3)
#convolution layer1
relu1=Activation_ReLU()
w1=np.random.rand(3,3,3,10)
b1=np.random.rand(1,1,1,10)
stride1=1
padding1=0
#convolution layer2
relu2=Activation_ReLU()
w2=np.random.rand(5,5,10,20)
b2=np.random.rand(1,1,1,20)
stride2=2
padding2=0
#convolution layer3
relu3=Activation_ReLU()
softmax=Activation_Softmax()
w3=np.random.rand(5,5,20,40)
b3=np.random.rand(1,1,1,40)
stride3=2
padding3=0

#convolution layer1 forward  
conv_output1=conv_forward(inputs,w1,b1,padding1,stride1)
relu1.forward(conv_output1,training=True)
print('shape of conv1 output ',relu1.output.shape)

#convolution layer2 forward 
conv_output2=conv_forward(relu.output,w2,b2,padding2,stride2)
relu2.forward(conv_output2,training=True)
print('shape of conv2 output ',relu2.output.shape)

#convolution layer3 forward 
conv_output3=conv_forward(relu2.output,w3,b3,padding3,stride3)
relu3.forward(conv_output3,training=True)
print('shape of conv3 output ',relu3.output.shape) 

#flatten the relu3 output to have shape [3,7*7*40] 
x=relu3.output.reshape(relu3.output.shape[0],-1)
print('shape after flattening ',x.shape)
#should be [3,1960]
print('shape match ',x.shape[1]==relu3.output.shape[1]*relu3.output.shape[2]*relu3.output.shape[3])
#feed x into softmax to get probability distributions 
softmax.forward(x,training=True)

shape of conv1 output  (3, 37, 37, 10)
shape of conv2 output  (3, 17, 17, 20)
shape of conv3 output  (3, 7, 7, 40)
shape after flattening  (3, 1960)
shape match  True


In [50]:
#max pooling layer example on 3d image
inputs=np.random.rand(15,15)
f=2
stride=2
#get inputs shape
(n_h_prev,n_w_prev)=inputs.shape

#define output shapes
n_h=int((n_h_prev-f)/stride)+1
n_w=int((n_w_prev-f)/stride)+1

#output
output=np.zeros((n_h,n_w))

for h in range(n_h):
    for w in range(n_w):
        #outputs 
        vert_start=h*stride
        vert_end=vert_start+f
        horiz_start=w*stride
        horiz_end=horiz_start+f

        inputs_slice=inputs[vert_start:vert_end,horiz_start:horiz_end]
        output[h,w]=np.max(inputs_slice)

print('inputs shape before max pooling ',inputs.shape)
print('inputs shape after max pooling ',output.shape)

inputs shape before max pooling  (15, 15)
inputs shape after max pooling  (7, 7)


In [47]:
#max pooling over batch of images
def max_pool(inputs,f,stride):
    #get shapes
    (m,n_h_prev,n_w_prev,n_c_prev)=inputs.shape
    
    #define output shape
    n_h=int((n_h_prev-f)/stride)+1
    n_w=int((n_w_prev-f)/stride)+1
    
    #output
    output=np.zeros((m,n_h,n_w,n_c_prev))

    for i in range(m):
        inputi=inputs[i]
        for h in range(n_h):
            for w in range(n_w):
                for c in range(n_c_prev):
                    #define vert/horiz start-end  
                    vert_start=h*stride
                    vert_end=vert_start+f
                    horiz_start=w*stride
                    horiz_end=horiz_start+f
                    #get image slice 
                    inputi_slice=inputi[vert_start:vert_end,horiz_start:horiz_end,c]
                    #max pooling on inputs[i] slice
                    output[i,h,w,c]=np.max(inputi_slice)

    return output


In [52]:
inputs=np.random.rand(3,15,15,3)
#hyperparameters shrinking by a factor of 2 
f=2
stride=2
output=max_pool(inputs,f,stride)

print('shape of inputs before max-pool ',inputs.shape)
print('shape of inputs after max-pool ',output.shape)

shape of inputs before max-pool  (3, 15, 15, 3)
shape of inputs after max-pool  (3, 7, 7, 3)


In [79]:
#CNN example forward pass ||something similar to LeNet-5  
inputs=np.random.rand(5,32,32,3)
#layer1 (conv,relu,max-pool)
relu1=Activation_ReLU()
w1=np.random.rand(5,5,3,6)
b1=np.random.rand(1,1,1,6)
stride1=1
padding1=0

#forward pass layer1(convolution + relu)
conv1_out=conv_forward(inputs,w1,b1,padding1,stride1)
relu1.forward(conv1_out,training=True)
#layer1 max pooling 
max_pool1=max_pool(relu1.output,f=2,stride=2)

#layer2
relu2=Activation_ReLU()
w2=np.random.rand(5,5,6,16)
b2=np.random.rand(1,1,1,16)
stride2=1
padding2=0
#layer2 forward pass(conv,relu,max-pool)
conv2_out=conv_forward(max_pool1,w2,b2,padding2,stride2)
relu2.forward(conv2_out,training=True)
print(relu2.output.shape)
max_pool2=max_pool(relu2.output,f=2,stride=2)

#flatten/reshape the max-pool2
x=max_pool2.reshape(max_pool2.shape[0],-1)

#fully-connected layer3
relu3=Activation_ReLU()
layer3=Layer_Dense(400,120)
#forward pass layer3(linear-layer,relu)
layer3.forward(x,training=True)
relu3.forward(layer3.output,training=True)

#fully-connected layer4
layer4=Layer_Dense(120,84)
relu4=Activation_ReLU()
layer4.forward(relu3.output,training=True)
relu4.forward(layer4.output,training=True)

#fully-connected layer5
layer5=Layer_Dense(84,10)
relu5=Activation_ReLU()
layer5.forward(relu4.output,training=True)
relu5.forward(layer5.output,training=True)


#softmax probability distribution 
softmax=Activation_Softmax()
softmax.forward(relu5.output,training=True)

print('shape of inputs ',inputs.shape)
print('shape of 1st layer max-out ',max_pool1.shape) 
print('shape of 2st layer max-out ',max_pool2.shape)
print('shape of inputs flattened ',x.shape)
print('shape of 3st fc relu  ',relu3.output.shape)
print('shape of 4th fc relu out  ',relu4.output.shape)
print('shape of 5th fc relu out  ',relu5.output.shape)
print('final softmax shape ',softmax.output.shape)


(5, 10, 10, 16)
shape of inputs  (5, 32, 32, 3)
shape of 1st layer max-out  (5, 14, 14, 6)
shape of 2st layer max-out  (5, 5, 5, 16)
shape of inputs flattened  (5, 400)
shape of 3st fc relu   (5, 120)
shape of 4th fc relu out   (5, 84)
shape of 5th fc relu out   (5, 10)
final softmax shape  (5, 10)
